# Bokeh

## ColumnDataSource

接受三种数据传入形式，分别为：

### Dict
data = {'x': [1,2,3,4], 'y': np.ndarray([10.0, 20.0, 30.0, 40.0])}<p>
source = ColumnDataSource(data)<p>    
### DataFrame Object
source = ColumnDataSource(df)<p>
### GroupBy Object
group = df.groupby(('colA', 'ColB'))<p>
source = ColumnDataSource(group)    
接着使用ColumnDataSource对返回数据进行处理生成source，在绘制图形的过程中，将source作为参数传入，此时通过列名便能获取到对应的数据。

## Bokeh中绘图的一般步骤
* 加载 bokeh 库，声明在 notebook 或 html 文件中显示或输出绘制的图表

* 绘制图表框架 figure()

* 在 figure 上绘制具体的图形，比如 circle，line，bar等

* 显示图片，show()

In [1]:
# 加载
import pandas as pd
import numpy as np
# 导入notebook绘图模块

from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.models import CustomJS, ColumnDataSource,HoverTool,Button, RadioButtonGroup, Select, Slider,CheckboxGroup, CustomJSFilter, CDSView, IndexFilter,RadioGroup,TextInput
# WidgetBox
from bokeh.core.properties import value
from bokeh.models.annotations import Span
from bokeh.layouts import column,gridplot,row

output_notebook()

Loading BokehJS ...

In [2]:
## Import data source'
dict_source={'LB':'lb.csv'}
for k,v in dict_source.items():
    globals()['df_'+k]=pd.read_csv('./data/bokeh/'+v)  

In [3]:
# make_dataset
lst_AnalyteName=['ALT','AST','BILI']
dict_color={'ALT':'red','AST':'olive','BILI':'blue'}
df_source= df_LB[df_LB['Subject']=='UAT012015']
df_source.head()

,Subject,SiteNumber,InstanceName,DataPageName,RecordPosition,AnalyteName,AnalyteValue,NumericValue,LabLow,LabHigh,LabUnits,LabFlag,BASELINE,RV_VISDT,days_from_baseline
441,UAT012015,UAT01,SCREENING,Lab,0,BILI,9.1,9.1,1.7,20.0,umol/L,0,7/2/2019,7/2/2019,0
442,UAT012015,UAT01,SCREENING,Lab,0,ALT,17,17.0,0.0,40.0,IU/L,0,7/2/2019,7/2/2019,0
443,UAT012015,UAT01,SCREENING,Lab,0,AST,15,15.0,0.0,45.0,IU/L,0,7/2/2019,7/2/2019,0
444,UAT012015,UAT01,UNS,Lab,0,BILI,17.1,17.1,1.7,20.0,umol/L,0,7/2/2019,7/3/2019,1
445,UAT012015,UAT01,UNS,Lab,0,ALT,23,23.0,0.0,40.0,IU/L,0,7/2/2019,7/3/2019,1


In [4]:
# figure
p = figure(width =800, height =300,toolbar_location="above",
           x_range=(0,50),
           y_range=(0,60),
           title="Patient XXXXX"
           )  

In [5]:
# figure style
p.xaxis.axis_label = "days_from_baseline"
p.yaxis.axis_label = "NumericValue" 

In [6]:
p.line(x=df_source[df_source['AnalyteName']=='ALT']['days_from_baseline'],
       y=df_source[df_source['AnalyteName']=='ALT']['NumericValue'],
       line_width=2,
       color=dict_color['ALT'])

GlyphRenderer(id='p1038', ...)

In [7]:
for test in lst_AnalyteName:
    p.line(df_source[df_source['AnalyteName']==test]['days_from_baseline'],
           df_source[df_source['AnalyteName']==test]['NumericValue'],
           x='days_from_baseline',
           y='NumericValue',
           line_width=2,
           color=dict_color[test], 
           legend_label = test
#             alpha=0.8,
           )

In [8]:
p.legend.location = "top_left"
p.legend.click_policy="hide"

In [9]:
show(p)

## bokeh的交互·主要有两种类型
一种是被动式（passive）的Inspector，一种是主动式（active）的Widgets。以下简单列举本次构建的APP涉及到的交互工具，并做简单说明介绍：

### 被动式（passive）的Inspector
HoverTool: 默认情况下，悬浮工具将生成“表格”类工具提示，其中每行包含标签及其关联信息。每个标签和对应信息构成一个元组，所有标签和信息构建成一个元组的列表作为参数进行传递。

检查器是基于当前光标位置对图像进行注释或报告图像信息的被动工具。

In [10]:
df_eDISH=df_LB[df_LB['AnalyteName'].isin(['AST','ALT','BILI'])]
df_eDISH=df_eDISH[df_eDISH['AnalyteValue'].notna()]
df_eDISH.head()

,Subject,SiteNumber,InstanceName,DataPageName,RecordPosition,AnalyteName,AnalyteValue,NumericValue,LabLow,LabHigh,LabUnits,LabFlag,BASELINE,RV_VISDT,days_from_baseline
3,UAT012002,UAT01,SCREENING,Lab,0,BILI,24.8,24.8,1.7,20.0,umol/L,+,12/27/2018,12/27/2018,0
4,UAT012002,UAT01,SCREENING,Lab,0,ALT,18,18.0,0.0,40.0,IU/L,0,12/27/2018,12/27/2018,0
5,UAT012002,UAT01,SCREENING,Lab,0,AST,17,17.0,0.0,45.0,IU/L,0,12/27/2018,12/27/2018,0
6,UAT012002,UAT01,CYCLE1_DAY8,Lab,0,BILI,27,27.0,1.7,20.0,umol/L,+,12/27/2018,1/16/2019,20
7,UAT012002,UAT01,CYCLE1_DAY8,Lab,0,ALT,15,15.0,0.0,40.0,IU/L,0,12/27/2018,1/16/2019,20


In [11]:
df_eDISH['VALUE/ULN']=df_eDISH['NumericValue']/df_eDISH['LabHigh']
df_eDISH.head()

,Subject,SiteNumber,InstanceName,DataPageName,RecordPosition,AnalyteName,AnalyteValue,NumericValue,LabLow,LabHigh,LabUnits,LabFlag,BASELINE,RV_VISDT,days_from_baseline,VALUE/ULN
3,UAT012002,UAT01,SCREENING,Lab,0,BILI,24.8,24.8,1.7,20.0,umol/L,+,12/27/2018,12/27/2018,0,1.240000
4,UAT012002,UAT01,SCREENING,Lab,0,ALT,18,18.0,0.0,40.0,IU/L,0,12/27/2018,12/27/2018,0,0.450000
5,UAT012002,UAT01,SCREENING,Lab,0,AST,17,17.0,0.0,45.0,IU/L,0,12/27/2018,12/27/2018,0,0.377778
6,UAT012002,UAT01,CYCLE1_DAY8,Lab,0,BILI,27,27.0,1.7,20.0,umol/L,+,12/27/2018,1/16/2019,20,1.350000
7,UAT012002,UAT01,CYCLE1_DAY8,Lab,0,ALT,15,15.0,0.0,40.0,IU/L,0,12/27/2018,1/16/2019,20,0.375000


In [12]:
df_eDISH_Peak=df_eDISH.groupby(['Subject','AnalyteName'])['VALUE/ULN'].max().reset_index()
df_eDISH_Peak.head()

,Subject,AnalyteName,VALUE/ULN
0,UAT012002,ALT,0.550000
1,UAT012002,AST,0.444444
2,UAT012002,BILI,1.780000
3,UAT012003,ALT,1.375000
4,UAT012003,AST,0.844444


In [13]:
def UnstackToFatDf(dfn,lst_key,lst_var,lv):
    df_temp=dfn.copy()
    df_temp=df_temp[lst_key+lst_var]
    df_temp=df_temp.set_index(lst_key)
    df_temp_unstack=df_temp.unstack(level=lv)
    df_temp_unstack.columns=df_temp_unstack.columns.droplevel()
    df_temp_unstack=df_temp_unstack.reset_index()
    return df_temp_unstack

df_eDISH_Peak_Unstack=UnstackToFatDf(df_eDISH_Peak,['Subject','AnalyteName'],['VALUE/ULN'],1)
df_eDISH_Peak_Unstack=df_eDISH_Peak_Unstack.rename(columns={'AST':'Peak_AST','ALT':'Peak_ALT','BILI':'Peak_BILI'})
# df_eDISH_Peak_Unstack.fillna(0,inplace=True)
df_eDISH_Peak_Unstack.head()

AnalyteName,Subject,Peak_ALT,Peak_AST,Peak_BILI
0,UAT012002,0.550,0.444444,1.780
1,UAT012003,1.375,0.844444,1.400
2,UAT012004,0.800,0.666667,0.605
3,UAT012006,0.700,0.511111,1.735
4,UAT012007,1.150,0.822222,1.435


In [14]:
# df_eDISH_Peak_Unstack.index.name = 'index'
source = ColumnDataSource(df_eDISH_Peak_Unstack)

In [15]:
hover = HoverTool(tooltips=[("Peak_AST", "@Peak_AST"),
                            ("Peak_BILI", "@Peak_BILI"),
                            ("Subject","@Subject")
                        ])

In [16]:
p1 = figure(width=800, height=300,toolbar_location="above",
            x_range=(0,2.5),
            y_range=(0,4.5),
            tools=[hover,'box_select,reset,wheel_zoom,pan,crosshair'],
            title="eDISH"
           )   # 注意这里书写方式;  hover它的作用是只是会显示出点的每个标签;crossshair是显示十字叉

In [17]:
# 如果不设置标签，就只写hover，例如 tools='hover,box_select,reset,wheel_zoom,pan,crosshair'
p1.circle(x = 'Peak_AST',
          y = 'Peak_BILI',
          source = source,
          size = 7,
#           alpha = 0.3        
         )

GlyphRenderer(id='p1122', ...)

In [18]:
p1.background_fill_color = "beige"    # 绘图空间背景颜色
p1.background_fill_alpha = 0.5        # 绘图空间背景透明度

p1.border_fill_color = "whitesmoke"    # 外边界背景颜色
p1.border_fill_alpha = 0.5             #透明度
p1.min_border_left = 80                # 外边界背景 - 左边宽度
p1.min_border_right = 80               # 外边界背景 - 右边宽度
p1.min_border_top = 10                 # 外边界背景 - 上宽度
p1.min_border_bottom = 10 

p1.xaxis.axis_label = "Peak_AST"
p1.xaxis.axis_line_width = 0.5
# p1.xaxis.bounds = (0, 3)

p1.yaxis.axis_label = "Peak_BILI"
# p1.yaxis.major_label_text_color = "orange"
# p1.yaxis.major_label_orientation = "vertical"

# p1.axis.minor_tick_in = 20     # 刻度往绘图区域内延伸长度；设置成负的就是往外边延伸了。
# p1.axis.minor_tick_out = 3   # 刻度往绘图区域外延伸长度

In [19]:
upper1 = Span(location=3,           # 设置位置，对应坐标值
             dimension='width',    # 设置方向，width为横向，height为纵向  
             line_color='olive', line_width=2,   # 设置线颜色、线宽
             line_dash = [8,4]
            )
upper2 = Span(location=2,dimension='height',line_color='firebrick', line_width=2,line_dash = [8,4])
p1.add_layout(upper1)
p1.add_layout(upper2)

In [20]:
show(p1)

### 主动式（active）的Widgets


窗口小部件是可以添加到bokeh应用程序的交互式控件，以便为可视化提供前端用户界面。窗口小部件可以驱动程序进行新的运算，更新绘图，甚至连接到其他程序。

CheckBoxGroup: 标准的复选框，支持多个指标同时选定

DatePicker: 更新日期

Button: 按钮，点击触发运算

Select: 单个选择工具，选项之间互斥

CustomJS callbacks：
    

In [21]:
# create some widgets
slider = Slider(start=0, end=10, value=1, step=.1, title="Slider")
button_group = RadioButtonGroup(labels=["Option 1", "Option 2", "Option 3"], active=0)
select = Select(title="Option:", value="foo", options=["foo", "bar", "baz", "quux"])
button_1 = Button(label="Button 1")
button_2 = Button(label="Button 2")
 
# put the results in a row
show(column(button_1, slider, button_group, select, button_2, width=300))
# show(row(button_1, slider, button_group, select, button_2))
# show(gridplot([[button_1, slider], [button_group, select],[button_2,]]))

### Examples

#### CustomJS for widgets

In [22]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import figure, show

x = [x*0.005 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(width=400, height=400, x_range=(0, 1), y_range=(0, 1))

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source), code="""
    const f = cb_obj.value
    const x = source.data.x
    const y = Array.from(x, (x) => Math.pow(x, f))
    source.data = { x, y }
""")

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
slider.js_on_change('value', callback)

layout = column(slider, plot)

show(layout)

#### CustomJS for selections

In [23]:
from random import random

from bokeh.layouts import row
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.plotting import figure, show

x = [random() for x in range(500)]
y = [random() for y in range(500)]

s1 = ColumnDataSource(data=dict(x=x, y=y))
p1 = figure(width=400, height=400, tools="lasso_select", title="Select Here")
p1.circle('x', 'y', source=s1, alpha=0.6)

s2 = ColumnDataSource(data=dict(x=[], y=[]))
p2 = figure(width=400, height=400, x_range=(0, 1), y_range=(0, 1),
            tools="", title="Watch Here")
p2.circle('x', 'y', source=s2, alpha=0.6)

s1.selected.js_on_change('indices', CustomJS(args=dict(s1=s1, s2=s2), code="""
        const inds = cb_obj.indices
        const d1 = s1.data
        const x = Array.from(inds, (i) => d1.x[i])
        const y = Array.from(inds, (i) => d1.y[i])
        s2.data = { x, y }
    """),
)

layout = row(p1, p2)

show(layout)

#### CustomJS for ranges

In [24]:
import numpy as np

from bokeh.layouts import row
from bokeh.models import BoxAnnotation, CustomJS
from bokeh.plotting import figure, show

N = 4000

x = np.random.random(size=N) * 100
y = np.random.random(size=N) * 100
radii = np.random.random(size=N) * 1.5
colors = np.array([(r, g, 150) for r, g in zip(50+2*x, 30+2*y)], dtype="uint8")

box = BoxAnnotation(left=0, right=0, bottom=0, top=0,
    fill_alpha=0.1, line_color='black', fill_color='black')

jscode = """
    box[%r] = cb_obj.start
    box[%r] = cb_obj.end
"""

p1 = figure(title='Pan and Zoom Here', x_range=(0, 100), y_range=(0, 100),
            tools='box_zoom,wheel_zoom,pan,reset', width=400, height=400)
p1.scatter(x, y, radius=radii, fill_color=colors, fill_alpha=0.6, line_color=None)

xcb = CustomJS(args=dict(box=box), code=jscode % ('left', 'right'))
ycb = CustomJS(args=dict(box=box), code=jscode % ('bottom', 'top'))

p1.x_range.js_on_change('start', xcb)
p1.x_range.js_on_change('end', xcb)
p1.y_range.js_on_change('start', ycb)
p1.y_range.js_on_change('end', ycb)

p2 = figure(title='See Zoom Window Here', x_range=(0, 100), y_range=(0, 100),
            tools='', width=400, height=400)
p2.scatter(x, y, radius=radii, fill_color=colors, fill_alpha=0.6, line_color=None)
p2.add_layout(box)

layout = row(p1, p2)

show(layout)

#### CustomJS for tools

In [25]:
from bokeh.events import SelectionGeometry
from bokeh.models import ColumnDataSource, CustomJS, Quad
from bokeh.plotting import figure, show

source = ColumnDataSource(data=dict(left=[], right=[], top=[], bottom=[]))

callback = CustomJS(args=dict(source=source), code="""
    const geometry = cb_obj.geometry
    const data = source.data

    // quad is forgiving if left/right or top/bottom are swappeed
    source.data = {
        left: data.left.concat([geometry.x0]),
        right: data.right.concat([geometry.x1]),
        top: data.top.concat([geometry.y0]),
        bottom: data.bottom.concat([geometry.y1])
    }
""")

p = figure(width=400, height=400, title="Select below to draw rectangles",
           tools="box_select", x_range=(0, 1), y_range=(0, 1))

# using Quad model directly to control (non)selection glyphs more carefully
quad = Quad(left='left', right='right',top='top', bottom='bottom',
            fill_alpha=0.3, fill_color='#009933')

p.add_glyph(source, quad, selection_glyph=quad, nonselection_glyph=quad)

p.js_on_event(SelectionGeometry, callback)

show(p)

#### CustomJS for hover tool

In [26]:
from bokeh.models import ColumnDataSource, CustomJS, HoverTool
from bokeh.plotting import figure, show

# define some points and a little graph between them
x = [2, 3, 5, 6, 8, 7]
y = [6, 4, 3, 8, 7, 5]
links = {
    0: [1, 2],
    1: [0, 3, 4],
    2: [0, 5],
    3: [1, 4],
    4: [1, 3],
    5: [2, 3, 4],
}

p = figure(width=400, height=400, tools="", toolbar_location=None, title='Hover over points')

source = ColumnDataSource({'x0': [], 'y0': [], 'x1': [], 'y1': []})
sr = p.segment(x0='x0', y0='y0', x1='x1', y1='y1', color='olive', alpha=0.6, line_width=3, source=source )
cr = p.circle(x, y, color='olive', size=30, alpha=0.4, hover_color='olive', hover_alpha=1.0)

# add a hover tool that sets the link data for a hovered circle
code = """
const links = %s
const data = {'x0': [], 'y0': [], 'x1': [], 'y1': []}
const indices = cb_data.index.indices
for (let i = 0; i < indices.length; i++) {
    const start = indices[i]
    for (let j = 0; j < links[start].length; j++) {
        const end = links[start][j]
        data['x0'].push(circle.data.x[start])
        data['y0'].push(circle.data.y[start])
        data['x1'].push(circle.data.x[end])
        data['y1'].push(circle.data.y[end])
    }
}
segment.data = data
""" % links

callback = CustomJS(args={'circle': cr.data_source, 'segment': sr.data_source}, code=code)
p.add_tools(HoverTool(tooltips=None, callback=callback, renderers=[cr]))

show(p)